# Load the data

In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
PROJ_ROOT = "./"

In [3]:
!ls {PROJ_ROOT}

Capstone_report.pdf     capstone_proposal.pdf   sub_final.csv
README.ipynb            capstone_proposal.tex   test.csv
README.md               sample_submission.csv   train.csv
capstone proposal.ipynb santendar-final.ipynb


In [4]:
TRAIN_DATA = os.path.join(PROJ_ROOT, "train.csv")
TEST_DATA = os.path.join(PROJ_ROOT, "test.csv")
SAMPLE_SUB = os.path.join(PROJ_ROOT, "sample_submission.csv")

## Read train data

In [5]:
%%time
train_df = pd.read_csv(TRAIN_DATA,index_col = 0)

CPU times: user 6.34 s, sys: 539 ms, total: 6.88 s
Wall time: 6.98 s


In [24]:
train_df.head()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
ID_code,,,,,,,,,,,,,,,,,,,,,
train_0,0,8.9255,-6.7863,11.9081,5.0930,11.4607,-9.2834,5.1187,18.6266,-4.9200,...,4.4354,3.9642,3.1364,1.6910,18.5227,-2.3978,7.8784,8.5635,12.7803,-1.0914
train_1,0,11.5006,-4.1473,13.8588,5.3890,12.3622,7.0433,5.6208,16.5338,3.1468,...,7.6421,7.7214,2.5837,10.9516,15.4305,2.0339,8.1267,8.7889,18.3560,1.9518
train_2,0,8.6093,-2.7457,12.0805,7.8928,10.5825,-9.0837,6.9427,14.6155,-4.9193,...,2.9057,9.7905,1.6704,1.6858,21.6042,3.1417,-6.5213,8.2675,14.7222,0.3965
train_3,0,11.0604,-2.1518,8.9522,7.1957,12.5846,-1.8361,5.8428,14.9250,-5.8609,...,4.4666,4.7433,0.7178,1.4214,23.0347,-1.2706,-2.9275,10.2922,17.9697,-8.9996
train_4,0,9.8369,-1.4834,12.8746,6.6375,12.2772,2.4486,5.9405,19.2514,6.2654,...,-1.4905,9.5214,-0.1508,9.1942,13.2876,-1.5121,3.9267,9.5031,17.9974,-8.8104


In [6]:
train_df.describe()

,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
count,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,...,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000,200000.000000
mean,0.100490,10.679914,-1.627622,10.715192,6.796529,11.078333,-5.065317,5.408949,16.545850,0.284162,...,3.234440,7.438408,1.927839,3.331774,17.993784,-0.142088,2.303335,8.908158,15.870720,-3.326537
std,0.300653,3.040051,4.050044,2.640894,2.043319,1.623150,7.863267,0.866607,3.418076,3.332634,...,4.559922,3.023272,1.478423,3.992030,3.135162,1.429372,5.454369,0.921625,3.010945,10.438015
min,0.000000,0.408400,-15.043400,2.117100,-0.040200,5.074800,-32.562600,2.347300,5.349700,-10.505500,...,-14.093300,-2.691700,-3.814500,-11.783400,8.694400,-5.261000,-14.209600,5.960600,6.299300,-38.852800
25%,0.000000,8.453850,-4.740025,8.722475,5.254075,9.883175,-11.200350,4.767700,13.943800,-2.317800,...,-0.058825,5.157400,0.889775,0.584600,15.629800,-1.170700,-1.946925,8.252800,13.829700,-11.208475
50%,0.000000,10.524750,-1.608050,10.580000,6.825000,11.108250,-4.833150,5.385100,16.456800,0.393700,...,3.203600,7.347750,1.901300,3.396350,17.957950,-0.172700,2.408900,8.888200,15.934050,-2.819550
75%,0.000000,12.758200,1.358625,12.516700,8.324100,12.261125,0.924800,6.003000,19.102900,2.937900,...,6.406200,9.512525,2.949500,6.205800,20.396525,0.829600,6.556725,9.593300,18.064725,4.836800
max,1.000000,20.315000,10.376800,19.353000,13.188300,16.671400,17.251600,8.447700,27.691800,10.151300,...,18.440900,16.716500,8.402400,18.281800,27.928800,4.272900,18.321500,12.000400,26.079100,28.500700


## Read test data

In [7]:
%%time
test_df = pd.read_csv(TEST_DATA, index_col = 0)
print("number of test samples = %d "%(len(test_df)))

test_df.head()

number of test samples = 200000 
CPU times: user 6.38 s, sys: 510 ms, total: 6.89 s
Wall time: 6.91 s


# Get X & y

In [8]:
target = 'target'
features = [ col for col in train_df.columns if col not in target ]

X = train_df[features]
y = train_df[target]
X.head(), y.head()

(           var_0   var_1    var_2   var_3    var_4   var_5   var_6    var_7  \
 ID_code                                                                       
 train_0   8.9255 -6.7863  11.9081  5.0930  11.4607 -9.2834  5.1187  18.6266   
 train_1  11.5006 -4.1473  13.8588  5.3890  12.3622  7.0433  5.6208  16.5338   
 train_2   8.6093 -2.7457  12.0805  7.8928  10.5825 -9.0837  6.9427  14.6155   
 train_3  11.0604 -2.1518   8.9522  7.1957  12.5846 -1.8361  5.8428  14.9250   
 train_4   9.8369 -1.4834  12.8746  6.6375  12.2772  2.4486  5.9405  19.2514   
 
           var_8   var_9  ...  var_190  var_191  var_192  var_193  var_194  \
 ID_code                  ...                                                
 train_0 -4.9200  5.7470  ...   4.4354   3.9642   3.1364   1.6910  18.5227   
 train_1  3.1468  8.0851  ...   7.6421   7.7214   2.5837  10.9516  15.4305   
 train_2 -4.9193  5.9525  ...   2.9057   9.7905   1.6704   1.6858  21.6042   
 train_3 -5.8609  8.2450  ...   4.4666   4.7433 

# Import required libraries

In [9]:


from matplotlib import pyplot as plt

from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import MinMaxScaler


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import StratifiedKFold


from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from lightgbm import LGBMModel
from xgboost import XGBClassifier



from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score


from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
import tensorflow as tf

random_state = 42
np.random.seed(random_state)


/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


# Data augumentation

In [10]:

# Ref - https://www.kaggle.com/kernels/scriptcontent/11630101/download
def augment(x,y,t=2):
    xs,xn = [],[]
    # Shuffle the feature values with target = 1 for 't' times 
    for i in range(t):
        # Get all the rows with y>0
        mask = y>0
        # Copy to temp array to not touch input array
        x1 = x[mask].copy()
        # Generate the sequence of indices to shuffle
        ids = np.arange(x1.shape[0])
        # Iterate through each column
        for c in range(x1.shape[1]):
            # Radomly shuffle the indices
            np.random.shuffle(ids)
            # Shuffle the column values 
            x1[:,c] = x1[ids][:,c]
        # Append the shuffled data in each iteration 
        xs.append(x1)

    # Shuffle the feature values with target = 0 for 't//2' times
    # t//2 because data is imbalance with more 0's. So add half the number of 1's 
    # compared to 0's as augumented data
    for i in range(t//2):
        
        # Get all the rows with y=0
        mask = y==0
        # Copy to temp array to not touch input array
        x1 = x[mask].copy()
        # Generate the sequence of indices to shuffle
        ids = np.arange(x1.shape[0])
        
        # Iterate through each column
        for c in range(x1.shape[1]):
            # Radomly shuffle the indices
            np.random.shuffle(ids)
            # Shuffle the column values 
            x1[:,c] = x1[ids][:,c]
            
        # Append the shuffled data in each iteration 
        xn.append(x1)
    
    # Stack up all the arrays row-wise
    xs = np.vstack(xs)
    xn = np.vstack(xn)
    
    # Create the augumented y array with 0's & 1's
    ys = np.ones(xs.shape[0])
    yn = np.zeros(xn.shape[0])
    
    # Stack up augumented array with original input array
    x = np.vstack([x,xs,xn])
    y = np.concatenate([y,ys,yn])
    return x,y

 # Helper methods 
 ## Plot ROC curve 

In [11]:

def plot_roc_curve(y_test, y_pred):
    fpr, tpr, threshold = roc_curve(y_test, y_pred, pos_label = 1)
    print("fpr = %s ,tpr = %s , threshold = %s " %(str(fpr),str(tpr),str(threshold)))
    #print(confusion_matrix(y_test,y_pred))
    roc_auc = auc(fpr, tpr)
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.scatter(fpr,tpr, s = 30, color = 'Red', marker = 'x')
    plt.legend(loc = 'lower right')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()
    return roc_auc_score(y_test,y_pred)




## Evaluate the model 

In [12]:
def evaluate_model(y_test, y_pred):
    print("====> MODEL EVALUATION: ")
    print("Accuray score = %f " %(accuracy_score(y_test,y_pred)))
    
    print("Precision score = %f " %(precision_score(y_test, y_pred, pos_label = 1)))
    
    print("Confusion Matrix: ")
    print(confusion_matrix(y_test, y_pred, labels = [1,0]))
    
    print("TP FN \n FP TN")
    
    plot_roc_curve(y_test,y_pred)
    
    return 
    

## Train and test a specific model 

In [13]:
def train_and_test_model(model, X_train, X_test, y_train, y_test):
    
    print("Training model with %d samples: " %len(X_train))
    print(model)
    
    # Fit the train samples  
    model.fit(X_train, y_train)
    print("Training completed!")
    
    print("Predicting the target for %d test samples:" %(len(X_test)))
      
    # Predict the target for test samples 
    y_pred = model.predict(X_test)
    print("Prediction complete!")
    
    evaluate_model(y_test, y_pred)
    return y_pred
    

## Grid search with roc_auc scorer & predict with best model  

In [14]:

def get_grid_best_estimator(X_train, y_train,base_estimator, params, scorer = None):
    # inputs of Grid search 
    
    # create cross validation sets 
    #cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.2, random_state = 50)
    cv_sets = StratifiedKFold(n_splits = 10, shuffle = False, random_state = 50)
    
    # Default scorer 
    if scorer is None: 
        scorer = make_scorer(roc_auc_score)
    
    # params
    
    # Grid search
    grid = GridSearchCV(base_estimator, params, scoring = scorer, cv = cv_sets )
    
    # fit 
    grid.fit(X_train,y_train)
    
    best_estimator  = grid.best_estimator_
    
    
    print("Best estimator : ")
    print(best_estimator)
    
    return best_estimator



def train_and_test_grid_search(X_train, X_test, y_train, y_test, base_estimator, params, scorer = None):
    
    
    best_estimator = get_grid_best_estimator(X_train, y_train, base_estimator, params, scorer)
    
    print("Predicting the target for %d test samples:" %(len(X_test)))
    y_pred = best_estimator.predict(X_test)
    
    print("Prediction complete!")
    
    evaluate_model(y_test, y_pred)
    return y_pred

# Stratified K fold

In [15]:
sk_folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)


# Light GBM 

In [16]:
# helper method to return LightGBM model object with the required params
def get_lgbm_obj():
    return LGBMModel( objective = "binary", 
                          scale_pos_weight = 1,
                          metric = "auc", 
                          boosting = 'gbdt', 
                          max_depth = -1, 
                          num_leaves = 13, 
                          learning_rate = 0.01, 
                          bagging_freq = 5, 
                          bagging_fraction = 0.4, 
                          feature_fraction = 0.05, 
                          min_data_in_leaf = 80, 
                          min_sum_heassian_in_leaf = 10, 
                          tree_learner = "serial", 
                          boost_from_average = False, 
                          bagging_seed = random_state, 
                          verbosity = 1, 
                          seed = random_state,
                         num_boost_round = 100000,
                         is_training_metric = True,
                         is_provide_training_metric=True)
                          

In [18]:
%%time
# list to store the validation AUC score of each fold
val_aucs = []

# Dataframe to store the predictions of each fold
predictions = pd.DataFrame(index=test_df.index)
predictions.reset_index(inplace=True)

# Iterate over each of the folds
for fold, (train_idx,valid_idx) in tqdm(enumerate(sk_folds.split(X,y))): 
    
    # Intialiaze the train sets and validation sets 
    X_train, y_train = None,None
    X_valid, y_valid = None,None
    
    # Extract the train and validation sets for current fold
    X_train,y_train = X.iloc[train_idx],y.iloc[train_idx]
    X_valid,y_valid = X.iloc[valid_idx],y.iloc[valid_idx]
    
    sub_iters = 5
    sub_y_pred, sub_y_valid_pred = 0,0
   
    # Run the agumentation, train & prediction steps for 5 times on the same fold
    for i in tqdm(range(sub_iters)):
    
        X_train_aug, y_train_aug = None, None
        X_train_aug_df, y_train_aug_df = None, None
        lgbm = None

        print("Augumentation for fold %d, sub_iter %d ..."%(fold,i), end="")
        X_train_aug, y_train_aug = augment(X_train.values,y_train.values)
        print("Done")

        X_train_aug_df = pd.DataFrame(X_train_aug,columns = features)
        y_train_aug_df = pd.DataFrame(y_train_aug,columns = [target])

        print("Fit start....", end="")
        lgbm = get_lgbm_obj()
        
        lgbm.fit(X_train_aug_df, y_train_aug_df, 
             eval_set = [(X_valid,y_valid)], # validate set for evaluation
             early_stopping_rounds = 3000, # early stopping
             eval_metric = 'auc', # evaluation metric for ealry stopping
             verbose = 1000)

        print("Done")
    
        # Predict the target for validation set 
        y_valid_pred = lgbm.predict(X_valid)
        sub_y_valid_pred += y_valid_pred
        
        print("Prediction start......", end = "")
        
        # Predict the target for actual test set 
        y_pred = lgbm.predict(test_df[features])
        sub_y_pred += y_pred
        print("Done")
       
    # compute the ROC AUC score on validation set    
    val_score = roc_auc_score(y_valid,sub_y_valid_pred)
    print("fold %d, sub_iter %d ... AUC = %f "%(fold,i,val_score), end="")
    # save the ROC AUC score of current fold
    val_aucs.append(val_score)
    # Save the predictions on the test data for this fold 
    predictions[f"fold_{fold}"] = sub_y_pred/sub_iters

Augumentation for fold 0, sub_iter 0 ...Done
Fit start....

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.881252
[2000]	valid_0's auc: 0.888551
[3000]	valid_0's auc: 0.893248
[4000]	valid_0's auc: 0.896158
[5000]	valid_0's auc: 0.897989
[6000]	valid_0's auc: 0.89899
[7000]	valid_0's auc: 0.899604
[8000]	valid_0's auc: 0.899895
[9000]	valid_0's auc: 0.900085
[10000]	valid_0's auc: 0.900214
[11000]	valid_0's auc: 0.900113
[12000]	valid_0's auc: 0.900152
[13000]	valid_0's auc: 0.900084
Early stopping, best iteration is:
[10046]	valid_0's auc: 0.900225
Done
Prediction start......Done
Augumentation for fold 0, sub_iter 1 ...Done
Fit start....Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.881041
[2000]	valid_0's auc: 0.888336
[3000]	valid_0's auc: 0.893023
[4000]	valid_0's auc: 0.895854
[5000]	valid_0's auc: 0.897972
[6000]	valid_0's auc: 0.898973
[7000]	valid_0's auc: 0.89966
[8000]	valid_0's auc: 0.899922
[9000]	valid_0's auc: 0.900127
[10000]	valid_0's auc: 0.900272

Augumentation for fold 1, sub_iter 0 ...Done
Fit start....

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


...

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.879496
[2000]	valid_0's auc: 0.886988
[3000]	valid_0's auc: 0.891783
[4000]	valid_0's auc: 0.894927
[5000]	valid_0's auc: 0.896931
[6000]	valid_0's auc: 0.898192
[7000]	valid_0's auc: 0.899001
[8000]	valid_0's auc: 0.899441
[9000]	valid_0's auc: 0.899683
[10000]	valid_0's auc: 0.899855
[11000]	valid_0's auc: 0.899893
[12000]	valid_0's auc: 0.899866
[13000]	valid_0's auc: 0.899844
[14000]	valid_0's auc: 0.89977
Early stopping, best iteration is:
[11333]	valid_0's auc: 0.89995
Done
Prediction start......Done
Augumentation for fold 3, sub_iter 1 ...Done
Fit start....Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.880202
[2000]	valid_0's auc: 0.886933
[3000]	valid_0's auc: 0.891653
[4000]	valid_0's auc: 0.89467
[5000]	valid_0's auc: 0.89668
[6000]	valid_0's auc: 0.898052
[7000]	valid_0's auc: 0.89884
[8000]	valid_0's auc: 0.899465
[9000]	valid_0's auc: 0.899853
[1

Augumentation for fold 4, sub_iter 0 ...Done
Fit start....

/Users/vthotakura/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.877104
[2000]	valid_0's auc: 0.884615
[3000]	valid_0's auc: 0.889428
[4000]	valid_0's auc: 0.892727
[5000]	valid_0's auc: 0.894769
[6000]	valid_0's auc: 0.895917
[7000]	valid_0's auc: 0.896503
[8000]	valid_0's auc: 0.896979
[9000]	valid_0's auc: 0.897153
[10000]	valid_0's auc: 0.897301
[11000]	valid_0's auc: 0.897341
[12000]	valid_0's auc: 0.897388
[13000]	valid_0's auc: 0.897311
[14000]	valid_0's auc: 0.897164
Early stopping, best iteration is:
[11904]	valid_0's auc: 0.897444
Done
Prediction start......Done
Augumentation for fold 4, sub_iter 1 ...Done
Fit start....Training until validation scores don't improve for 3000 rounds.
[1000]	valid_0's auc: 0.876969
[2000]	valid_0's auc: 0.88448
[3000]	valid_0's auc: 0.889263
[4000]	valid_0's auc: 0.892364
[5000]	valid_0's auc: 0.894461
[6000]	valid_0's auc: 0.895746
[7000]	valid_0's auc: 0.896372
[8000]	valid_0's auc: 0.896834
[9000]	valid_0's auc: 0.89700

### ROC_AUC scores of each fold

In [21]:
#lgbm.evals_result_
val_aucs

[0.9010536240727383,
 0.9014202562131672,
 0.9071433203631647,
 0.9005763341194364,
 0.8980476284065365]

In [22]:
from statistics import mean, median
print(f"Mean ROC-AUC score across folds: {np.mean(val_aucs)}")

Mean ROC-AUC score across folds: 0.9016482326350086


In [23]:
predictions.head()

,ID_code,fold_0,fold_1,fold_2,fold_3,fold_4
0,test_0,0.119674,0.113514,0.154817,0.128497,0.150235
1,test_1,0.389530,0.232405,0.268074,0.280413,0.281199
2,test_2,0.243543,0.247582,0.241249,0.254615,0.203856
3,test_3,0.262631,0.265267,0.303822,0.332081,0.285003
4,test_4,0.064820,0.040867,0.063159,0.067750,0.062975


In [24]:
predictions.columns

Index(['ID_code', 'fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4'], dtype='object')

In [25]:
pred_cols = ['fold_0', 'fold_1', 'fold_2', 'fold_3', 'fold_4']

# Final prediction score
### as a mean of prediction scores from each fold

In [26]:
predictions['target'] = np.mean(predictions[[col for col in predictions.columns if col not in ['ID_code', 'target']]].values, axis=1)


In [27]:
# Round the predictions to 0 or 1
predictions['target_round'] = np.round(predictions['target'])

# Create submission file

In [28]:
# Read sample submission file
sample_sub = pd.read_csv(SAMPLE_SUB)

In [29]:
sample_sub.head()

,ID_code,target
0,test_0,0
1,test_1,0
2,test_2,0
3,test_3,0
4,test_4,0


In [30]:
# update the predictions in submission dataframe
sample_sub['target'] = predictions['target']

In [31]:
sample_sub.describe()

,target
count,200000.000000
mean,0.117144
std,0.168520
min,0.000088
25%,0.017692
50%,0.049209
75%,0.137730
max,0.998791


sample_sub['target

In [32]:
# Write to csv
sample_sub.to_csv("./sub_final.csv",index=False)

#### Save all predictions

In [33]:
sample_sub_round = sample_sub.copy()

In [34]:
predictions.to_csv("./predictions_final.csv")

In [35]:
predictions.head()

,ID_code,fold_0,fold_1,fold_2,fold_3,fold_4,target,target_round
0,test_0,0.119674,0.113514,0.154817,0.128497,0.150235,0.133347,0.0
1,test_1,0.389530,0.232405,0.268074,0.280413,0.281199,0.290324,0.0
2,test_2,0.243543,0.247582,0.241249,0.254615,0.203856,0.238169,0.0
3,test_3,0.262631,0.265267,0.303822,0.332081,0.285003,0.289761,0.0
4,test_4,0.064820,0.040867,0.063159,0.067750,0.062975,0.059914,0.0


In [36]:
sample_sub_round['target'] = predictions['target_round']

In [37]:
sample_sub_round.to_csv("./sub_final_round.csv", index=False)